In [1]:
import mediapipe as mp
import cv2

In [4]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np

def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변환
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [3]:
# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False,        # 입력 이미지가 정적 이미지인지, 비디오 스트림 프레임인지 설정(False인 경우 비디오 스트림으로 처리)
    model_complexity=1,             # 0,1,2 의 순으로 정확도가 올라가는 대신 속도가 느림
    smooth_landmarks=True,          # 프레임간의 랜드 마크 위치를 부드럽게 처리할 지 여부 결정
    enable_segmentation=True,       # 포즈 검출과 인물 마스크를 생성할지 여부 결정. 배경 제거 등의 용도로 사용
    smooth_segmentation=True,       # 인물 마스크의 경계를 부드럽게 처리
    min_detection_confidence=0.5,   # 사람 검출 최소 신뢰도 임계값
    min_tracking_confidence=0.5     # 포즈 추적의 신뢰도 임계값
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while True:
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.segmentation_mask is not None:
        # 세그멘테이션 마스크를 이미지 크기에 맞게 조정
        segmentation_mask = results.segmentation_mask
        segmentation_mask = (segmentation_mask > 0.1)  # 임계값 적용
        segmentation_mask = segmentation_mask.astype(float)
        
        # 세그멘테이션 마스크를 3채널로 복제하여 컬러 이미지와 동일한 차원으로 만듦
        mask_three_channel = cv2.merge([segmentation_mask, segmentation_mask, segmentation_mask])
        
        # 마스크를 적용하여 배경을 제거한 이미지 생성
        # foreground = cv2.multiply(mask_three_channel, image.astype(float))
        # cv2.imshow('Foreground', foreground.astype('uint8'))

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()